<a href="https://colab.research.google.com/github/srinakasam19/CodEvo-Solutions/blob/main/bitcoin_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Here we are importing Library which are required
import pandas as pd
import numpy as np
import math
import datetime as dt

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
# Loading Dataset and remainig its columns

data=pd.read_csv('/content/BTC-USD.csv')
data = data.rename(columns={'Date': 'date','Open':'open','High':'high','Low':'low','Close':'close',
                                'Adj Close':'adj_close','Volume':'volume'})

In [ ]:
data.describe()

,open,high,low,close,adj_close,volume
count,2530.000000,2530.000000,2530.000000,2530.000000,2530.000000,2.530000e+03
mean,8581.208790,8827.542899,8314.356161,8599.314657,8599.314657,1.325395e+10
std,12672.856486,13066.982740,12225.142520,12695.366642,12695.366642,1.979314e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,572.634476,577.809250,566.011734,572.849243,572.849243,7.178582e+07
50%,5077.713868,5282.096680,4987.065674,5093.062745,5093.062745,4.299460e+09
75%,9469.311767,9654.489258,9262.651612,9477.050537,9477.050537,2.066714e+10
max,63523.753906,64863.097656,62208.964844,63503.457031,63503.457031,3.509679e+11


In [ ]:
data.isnull().sum()

,0
date,0
open,4
high,4
low,4
close,4
adj_close,4
volume,4


In [ ]:
data['date'] = pd.to_datetime(data.date)
data.head()

,date,open,high,low,close,adj_close,volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0


In [ ]:
print("Starting date: ",data.iloc[0][0])
print("Ending date: ", data.iloc[-1][0])
print("Duration: ", data.iloc[-1][0]-data.iloc[0][0])

Starting date:  2014-09-17 00:00:00
Ending date:  2021-08-24 00:00:00
Duration:  2533 days 00:00:00


<ipython-input-6-13b5327963b2>:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-6-13b5327963b2>:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-6-13b5327963b2>:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [ ]:
y_2014 = data.loc[(data['date'] >= '2014-01-01')
                     & (data['date'] < '2015-01-01')]

y_2014.drop(y_2014[['adj_close','volume']],axis=1)

,date,open,high,low,close
0,2014-09-17,465.864014,468.174011,452.421997,457.334015
1,2014-09-18,456.859985,456.859985,413.104004,424.440002
2,2014-09-19,424.102997,427.834991,384.532013,394.795990
3,2014-09-20,394.673004,423.295990,389.882996,408.903992
4,2014-09-21,408.084991,412.425995,393.181000,398.821014
...,...,...,...,...,...
101,2014-12-27,327.583008,328.911011,312.630005,315.863007
102,2014-12-28,316.160004,320.028015,311.078003,317.239014
103,2014-12-29,317.700989,320.266998,312.307007,312.670013
104,2014-12-30,312.718994,314.808990,309.372986,310.737000


In [ ]:
monthvise= y_2014.groupby(y_2014['date'].dt.strftime('%B'))[['open','close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

,open,close
date,,
January,NaN,NaN
February,NaN,NaN
March,NaN,NaN
April,NaN,NaN
May,NaN,NaN
June,NaN,NaN
July,NaN,NaN
August,NaN,NaN
September,412.654003,407.182428


In [ ]:
import plotly.io as pio
pio.renderers.default = 'notebook'  # For Jupyter
# pio.renderers.default = 'colab'  # For Google Colab


In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Ensure the renderer is set for Google Colab
pio.renderers.default = 'colab'

# Example: Load data (replace with your actual data loading method)
# For demonstration, replace this with your actual data loading code
data = {
    'date': pd.date_range(start='2014-01-01', periods=12, freq='M'),
    'high': [120, 130, 125, 140, 135, 150, 160, 155, 145, 140, 130, 125],
    'low': [100, 110, 105, 120, 115, 130, 140, 135, 125, 120, 110, 105]
}
y_2014 = pd.DataFrame(data)

# Convert 'date' column to datetime if not already
y_2014['date'] = pd.to_datetime(y_2014['date'])

# Define the new order of months
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Group by month and calculate high and low prices
monthvise_high = y_2014.groupby(y_2014['date'].dt.strftime('%B'))['high'].max()
monthvise_high = monthvise_high.reindex(new_order)

monthvise_low = y_2014.groupby(y_2014['date'].dt.strftime('%B'))['low'].min()
monthvise_low = monthvise_low.reindex(new_order)

# Create the plot
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock High Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock Low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(
    barmode='group',
    title='Monthwise High and Low Stock Prices'
)

fig.show()


In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from itertools import cycle

# Set the renderer for Google Colab
pio.renderers.default = 'colab'

# Print column names to diagnose
print("Column names before renaming:", y_2014.columns)

# Remove any leading/trailing spaces
y_2014.columns = y_2014.columns.str.strip()

# Print column names after stripping spaces
print("Column names after stripping spaces:", y_2014.columns)

# Rename columns if necessary
y_2014.rename(columns={
    'date': 'Date',
    'open': 'Open',
    'close': 'Close',
    'high': 'High',
    'low': 'Low',
    'adjclose': 'Adj Close',
    'volume': 'Volume'
}, inplace=True)

# Print column names after renaming
print("Column names after renaming:", y_2014.columns)

# Reshape the DataFrame to long format with existing columns
y_long = y_2014.melt(id_vars='Date', value_vars=['High', 'Low'],
                     var_name='Parameter', value_name='Stock value')

# Define a cycle for legend names
names = cycle(['Stock High Price', 'Stock Low Price'])

# Create the line plot using Plotly Express
fig = px.line(y_long, x='Date', y='Stock value', color='Parameter',
              labels={'Date': 'Date', 'Stock value': 'Stock value'},
              title='Stock Analysis Chart')

# Update legend names
fig.for_each_trace(lambda t: t.update(name=next(names)))

# Update layout
fig.update_layout(
    font_size=15,
    font_color='black',
    legend_title_text='Stock Parameters'
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Show plot
fig.show()


Column names before renaming: Index(['Date', 'High', 'Low'], dtype='object')
Column names after stripping spaces: Index(['Date', 'High', 'Low'], dtype='object')
Column names after renaming: Index(['Date', 'High', 'Low'], dtype='object')


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:


# Define a cycle for legend names
names = cycle(['Stock Open Price', 'Stock Close Price', 'Stock High Price', 'Stock Low Price'])

# Create the line plot using Plotly Express
fig = px.line(y_long, x='Date', y='Stock value', color='Parameter',
              labels={'Date': 'Date', 'Stock value': 'Stock value'},
              title='Stock Analysis Chart')

# Update legend names
fig.for_each_trace(lambda t: t.update(name=next(names)))

# Update layout
fig.update_layout(
    font_size=15,
    font_color='black',
    legend_title_text='Stock Parameters'
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Show plot
fig.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
import pandas as pd

# Convert dictionary to DataFrame
data_df = pd.DataFrame(data)

# Ensure 'date' column is in datetime format
data_df['date'] = pd.to_datetime(data_df['date'])

# Print column names to verify
print("Columns in the DataFrame:", data_df.columns)

# Filter data for the year 2020
y_2021 = data_df.loc[(data_df['date'] >= '2020-01-01') & (data_df['date'] < '2021-01-01')]

# Drop columns if they exist
columns_to_drop = ['adj_close', 'volume']
existing_columns_to_drop = [col for col in columns_to_drop if col in y_2020.columns]
y_2021 = y_2021.drop(columns=existing_columns_to_drop)

print(y_2021.head())


Columns in the DataFrame: Index(['date', 'high', 'low'], dtype='object')
Empty DataFrame
Columns: [date, high, low]
Index: []
